### RULE 

第 0 列大于平均值

### 训练信息

In [1]:
# 训练信息
rule_model_train_time      = ""
log_path                   = ""          
model_path                 = ""
train_data_path            = ""

gan_train_time             = ""
gan_path                   = ""

### 全局参数

In [4]:
COL_1 = 0

### GT 中的 error rate

In [7]:
import numpy as np
import pandas as pd

GROUND_TRUTH_PATH = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/data/Ticket/Ticket.csv"

def verify_ground_truth_rule():
    file = np.array(pd.read_csv(GROUND_TRUTH_PATH, sep=','))
    target = file[:,0]
    target_mean =  np.mean(target)
    print("mean:", target_mean)
    col_1 = COL_1
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] > target_mean:
                error_list.append((i+2, j+2))
                t -= 1
                if t < 0:
                    isOk = False
                    count += 1
                    break
    print("error: {}, error rate:{}".format(count, count / len(file)))
    return count

error_GT = verify_ground_truth_rule()

mean: 22.8245
error: 922, error rate:0.461


### 给数据添加标签

In [10]:
X_OUTPUT = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/data/Ticket/Ticket_rulemodel.csv"
Y_OUTPUT = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/data/Ticket/Ticket_rulemodel_labels.csv"

def generate_data():
    file = np.array(pd.read_csv(GROUND_TRUTH_PATH, sep=','))
    target = file[:,0]
    target_mean =  np.mean(target)
    print("mean:", target_mean)
    positive = []
    labels = []
    negative = []
    col_1 = COL_1
    error = 0
    for i, _ in enumerate(file):
        isOK = True
        for j, _ in enumerate(file):
            if file[i][col_1] > target_mean:
                negative.append(file[i])
                labels.append([1])
                error += 1
                isOK = False
                break
        if isOK:
            positive.append(file[i])
            labels.append([0])

    X = np.array(positive + negative)
    y = np.array(labels)
    X_out = pd.DataFrame(X)
    y_out = pd.DataFrame(y)
    X_out.to_csv(X_OUTPUT, index=False, sep=',')
    y_out.to_csv(Y_OUTPUT, index=False, sep=',')
    print("输出完成！Xshape:{}, yshape:{} error: {}, error rate:{}".format(X.shape, y.shape, error, error / len(X)))
    return error

error_GD = generate_data()

mean: 22.8245
输出完成！Xshape:(2000, 14), yshape:(2000, 1) error: 922, error rate:0.461


In [11]:
assert(error_GT == error_GD)

### 验证 Fake Data

In [6]:
# >, >
OUTPUT_TMP_PATH = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/samples/Ticket/Ticket_OI_11_00_fake.csv"
OUTPUT_PATH = ""
ORIGINAL_PATH = "/Users/luminshen/Desktop/模型/original gan/Ticket/Ticket_OI_11_00_fake.csv"

def verify_rule(path, ratio, c1, c2, name):
    if ratio == -1:
        file = list(np.array(pd.read_csv(path, sep=',')))
    else:
        file = list(np.array(pd.read_csv(path, sep=';')))
    col_1 = c1
    col_2 = c2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] > file[j][col_1] and file[i][col_2] > file[j][col_2]:
                t -= 1
                if t < 0:
                    count += 1

    error_rate = count / len(file)**2
    if ratio == -1:
        print("ground truth: error: {:8.0f}, error rate:{:8.6f}".format(count, count / len(file)**2))
        return count / len(file)**2    
    
    else:    
        if error_rate - ratio > 0:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:+{:8.6f}".format(name, count, error_rate, error_rate - ratio))
        else:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:{:8.6f}".format(name, count, error_rate, error_rate - ratio))

        return error_rate - ratio, count


ratio = verify_rule(GROUND_TRUTH_PATH, -1, 0, 9, "")
verify_rule(OUTPUT_TMP_PATH, ratio, 0, 9, "fake")
verify_rule(ORIGINAL_PATH, ratio, 0, 9, "original")
print("------------------------------------------------")

count_fake = 0
count_original = 0
sum_fake = 0
sum_original = 0

for c1 in range(10):
    for c2 in range(10):
        if c1 == c2:
            continue
        ratio = verify_rule(GROUND_TRUTH_PATH, -1, c1, c2 , "")
        diff_fake, error_new = verify_rule(OUTPUT_TMP_PATH, ratio, c1, c2, "fake")
        diff_original, error_original = verify_rule(ORIGINAL_PATH, ratio, c1, c2, "original")
        sum_fake += abs(error_new)
        sum_original += abs(error_original)
        if abs(diff_fake) > abs(diff_original):
            count_original += 1
        else:
            count_fake += 1
        print("------------------------------------------------ {}:{}".format(count_fake, count_original))

print("fake: {}, original: {}, ratio: {}, fake_diff: {}. original_diff: {}".format(count_fake, count_original, count_fake/count_original, sum_fake, sum_original))



ground truth: error:   460476, error rate:0.115119
fake        : error:   405529, error rate:0.101382, diff:-0.013737
original    : error:        0, error rate:0.000000, diff:-0.115119
------------------------------------------------
ground truth: error:  1130253, error rate:0.282563
fake        : error:  1222664, error rate:0.305666, diff:+0.023103
original    : error:  1155810, error rate:0.288953, diff:+0.006389
------------------------------------------------ 0:1
ground truth: error:  1130253, error rate:0.282563
fake        : error:   991020, error rate:0.247755, diff:-0.034808
original    : error:  1130966, error rate:0.282741, diff:+0.000178
------------------------------------------------ 0:2
ground truth: error:   807720, error rate:0.201930
fake        : error:   177263, error rate:0.044316, diff:-0.157614
original    : error:   588467, error rate:0.147117, diff:-0.054813
------------------------------------------------ 0:3
ground truth: error:   852318, error rate:0.213080
f

original    : error:  1639148, error rate:0.409787, diff:+0.262576
------------------------------------------------ 15:19
ground truth: error:   570668, error rate:0.142667
fake        : error:   821354, error rate:0.205339, diff:+0.062672
original    : error:  1128850, error rate:0.282212, diff:+0.139545
------------------------------------------------ 16:19
ground truth: error:   446198, error rate:0.111549
fake        : error:     1992, error rate:0.000498, diff:-0.111051
original    : error:        0, error rate:0.000000, diff:-0.111549
------------------------------------------------ 17:19
ground truth: error:   852318, error rate:0.213080
fake        : error:   817036, error rate:0.204259, diff:-0.008821
original    : error:   736491, error rate:0.184123, diff:-0.028957
------------------------------------------------ 18:19
ground truth: error:   823965, error rate:0.205991
fake        : error:   528490, error rate:0.132123, diff:-0.073869
original    : error:   810368, error rat

ground truth: error:  1086730, error rate:0.271682
fake        : error:   830963, error rate:0.207741, diff:-0.063942
original    : error:   633420, error rate:0.158355, diff:-0.113327
------------------------------------------------ 35:34
ground truth: error:   703387, error rate:0.175847
fake        : error:   339724, error rate:0.084931, diff:-0.090916
original    : error:   588162, error rate:0.147040, diff:-0.028806
------------------------------------------------ 35:35
ground truth: error:  1625096, error rate:0.406274
fake        : error:  1468375, error rate:0.367094, diff:-0.039180
original    : error:  1425238, error rate:0.356310, diff:-0.049965
------------------------------------------------ 36:35
ground truth: error:   558338, error rate:0.139585
fake        : error:   465850, error rate:0.116462, diff:-0.023122
original    : error:        0, error rate:0.000000, diff:-0.139585
------------------------------------------------ 37:35
ground truth: error:  1003457, error rat

In [8]:
# =, !=
OUTPUT_TMP_PATH = "/Users/luminshen/Desktop/模型/new gan/T-3>9>/Ticket_OI_11_00_fake.csv"
OUTPUT_PATH = ""
ORIGINAL_PATH = "/Users/luminshen/Desktop/模型/original gan/Ticket/Ticket_OI_11_00_fake.csv"

def verify_rule(path, ratio, c1, c2, name):
    if ratio == -1:
        file = list(np.array(pd.read_csv(path, sep=',')))
    else:
        file = list(np.array(pd.read_csv(path, sep=';')))
    col_1 = c1
    col_2 = c2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] == file[j][col_1] and file[i][col_2] != file[j][col_2]:
                t -= 1
                if t < 0:
                    count += 1

    error_rate = count / len(file)**2
    if ratio == -1:
        print("ground truth: error: {:8.0f}, error rate:{:8.6f}".format(count, count / len(file)**2))
        return count / len(file)**2    
    
    else:    
        if error_rate - ratio > 0:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:+{:8.6f}".format(name, count, error_rate, error_rate - ratio))
        else:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:{:8.6f}".format(name, count, error_rate, error_rate - ratio))

        return error_rate - ratio, count



ratio = verify_rule(GROUND_TRUTH_PATH, -1, 0, 9, "")
verify_rule(OUTPUT_TMP_PATH, ratio, 0, 9, "fake")
verify_rule(ORIGINAL_PATH, ratio, 0, 9, "original")
print("------------------------------------------------")

count_fake = 0
count_original = 0
sum_fake = 0
sum_original = 0

for c1 in range(10):
    for c2 in range(10):
        if c1 == c2:
            continue
        ratio = verify_rule(GROUND_TRUTH_PATH, -1, c1, c2 , "")
        diff_fake, error_new = verify_rule(OUTPUT_TMP_PATH, ratio, c1, c2, "fake")
        diff_original, error_original = verify_rule(ORIGINAL_PATH, ratio, c1, c2, "original")
        sum_fake += abs(error_new)
        sum_original += abs(error_original)
        if abs(diff_fake) > abs(diff_original):
            count_original += 1
        else:
            count_fake += 1
        print("------------------------------------------------ {}:{}".format(count_fake, count_original))

print("fake: {}, original: {}, ratio: {}, fake_diff: {}. original_diff: {}".format(count_fake, count_original, count_fake/count_original, sum_fake, sum_original))


ground truth: error:   236086, error rate:0.059021
fake        : error:    37930, error rate:0.009482, diff:-0.049539
original    : error:        0, error rate:0.000000, diff:-0.059021
------------------------------------------------
ground truth: error:        0, error rate:0.000000
fake        : error:    98060, error rate:0.024515, diff:+0.024515
original    : error:   157630, error rate:0.039407, diff:+0.039407
------------------------------------------------ 1:0
ground truth: error:        0, error rate:0.000000
fake        : error:   102022, error rate:0.025506, diff:+0.025506
original    : error:   164314, error rate:0.041078, diff:+0.041078
------------------------------------------------ 2:0
ground truth: error:        0, error rate:0.000000
fake        : error:   101658, error rate:0.025414, diff:+0.025414
original    : error:   186182, error rate:0.046545, diff:+0.046545
------------------------------------------------ 3:0
ground truth: error:   453224, error rate:0.113306
f

original    : error:   180694, error rate:0.045173, diff:-0.071208
------------------------------------------------ 26:8
ground truth: error:   452424, error rate:0.113106
fake        : error:   186044, error rate:0.046511, diff:-0.066595
original    : error:   181492, error rate:0.045373, diff:-0.067733
------------------------------------------------ 27:8
ground truth: error:   236086, error rate:0.059021
fake        : error:    62844, error rate:0.015711, diff:-0.043311
original    : error:        0, error rate:0.000000, diff:-0.059021
------------------------------------------------ 28:8
ground truth: error:   147346, error rate:0.036837
fake        : error:   150172, error rate:0.037543, diff:+0.000706
original    : error:   162076, error rate:0.040519, diff:+0.003682
------------------------------------------------ 29:8
ground truth: error:   147346, error rate:0.036837
fake        : error:   146748, error rate:0.036687, diff:-0.000150
original    : error:   163638, error rate:0.

ground truth: error:        0, error rate:0.000000
fake        : error:     6586, error rate:0.001646, diff:+0.001646
original    : error:    10322, error rate:0.002580, diff:+0.002580
------------------------------------------------ 54:15
ground truth: error:        0, error rate:0.000000
fake        : error:     6404, error rate:0.001601, diff:+0.001601
original    : error:     9594, error rate:0.002399, diff:+0.002399
------------------------------------------------ 55:15
ground truth: error:        0, error rate:0.000000
fake        : error:     6812, error rate:0.001703, diff:+0.001703
original    : error:    10428, error rate:0.002607, diff:+0.002607
------------------------------------------------ 56:15
ground truth: error:        2, error rate:0.000000
fake        : error:     2706, error rate:0.000677, diff:+0.000676
original    : error:        0, error rate:0.000000, diff:-0.000000
------------------------------------------------ 56:16
ground truth: error:        0, error rat

In [10]:
# >, !=
OUTPUT_TMP_PATH = "/Users/luminshen/Desktop/模型/new gan/T-3>9>/Ticket_OI_11_00_fake.csv"
OUTPUT_PATH = ""
ORIGINAL_PATH = "/Users/luminshen/Desktop/模型/original gan/Ticket/Ticket_OI_11_00_fake.csv"

def verify_rule(path, ratio, c1, c2, name):
    if ratio == -1:
        file = list(np.array(pd.read_csv(path, sep=',')))
    else:
        file = list(np.array(pd.read_csv(path, sep=';')))
    col_1 = c1
    col_2 = c2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] > file[j][col_1] and file[i][col_2] != file[j][col_2]:
                t -= 1
                if t < 0:
                    count += 1

    error_rate = count / len(file)**2
    if ratio == -1:
        print("ground truth: error: {:8.0f}, error rate:{:8.6f}".format(count, count / len(file)**2))
        return count / len(file)**2    
    
    else:    
        if error_rate - ratio > 0:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:+{:8.6f}".format(name, count, error_rate, error_rate - ratio))
        else:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:{:8.6f}".format(name, count, error_rate, error_rate - ratio))

        return error_rate - ratio, count



ratio = verify_rule(GROUND_TRUTH_PATH, -1, 0, 9, "")
verify_rule(OUTPUT_TMP_PATH, ratio, 0, 9, "fake")
verify_rule(ORIGINAL_PATH, ratio, 0, 9, "original")
print("------------------------------------------------")

count_fake = 0
count_original = 0
sum_fake = 0
sum_original = 0

for c1 in range(10):
    for c2 in range(10):
        if c1 == c2:
            continue
        ratio = verify_rule(GROUND_TRUTH_PATH, -1, c1, c2 , "")
        diff_fake, error_new = verify_rule(OUTPUT_TMP_PATH, ratio, c1, c2, "fake")
        diff_original, error_original = verify_rule(ORIGINAL_PATH, ratio, c1, c2, "original")
        sum_fake += abs(error_new)
        sum_original += abs(error_original)
        if abs(diff_fake) > abs(diff_original):
            count_original += 1
        else:
            count_fake += 1
        print("------------------------------------------------ {}:{}".format(count_fake, count_original))

print("fake: {}, original: {}, ratio: {}, fake_diff: {}. original_diff: {}".format(count_fake, count_original, count_fake/count_original, sum_fake, sum_original))



ground truth: error:   896321, error rate:0.224080
fake        : error:   785897, error rate:0.196474, diff:-0.027606
original    : error:        0, error rate:0.000000, diff:-0.224080
------------------------------------------------
ground truth: error:  1763357, error rate:0.440839
fake        : error:  1836608, error rate:0.459152, diff:+0.018313
original    : error:  1842158, error rate:0.460539, diff:+0.019700
------------------------------------------------ 1:0
ground truth: error:  1763357, error rate:0.440839
fake        : error:  1839429, error rate:0.459857, diff:+0.019018
original    : error:  1823419, error rate:0.455855, diff:+0.015015
------------------------------------------------ 1:1
ground truth: error:  1763357, error rate:0.440839
fake        : error:  1854704, error rate:0.463676, diff:+0.022837
original    : error:  1813432, error rate:0.453358, diff:+0.012519
------------------------------------------------ 1:2
ground truth: error:  1689684, error rate:0.422421
f

original    : error:  1903196, error rate:0.475799, diff:+0.034960
------------------------------------------------ 22:12
ground truth: error:  1763357, error rate:0.440839
fake        : error:  1900160, error rate:0.475040, diff:+0.034201
original    : error:  1882657, error rate:0.470664, diff:+0.029825
------------------------------------------------ 22:13
ground truth: error:   896321, error rate:0.224080
fake        : error:   773440, error rate:0.193360, diff:-0.030720
original    : error:        0, error rate:0.000000, diff:-0.224080
------------------------------------------------ 23:13
ground truth: error:  1689684, error rate:0.422421
fake        : error:  1851227, error rate:0.462807, diff:+0.040386
original    : error:  1816082, error rate:0.454020, diff:+0.031600
------------------------------------------------ 23:14
ground truth: error:  1689684, error rate:0.422421
fake        : error:  1812264, error rate:0.453066, diff:+0.030645
original    : error:  1839154, error rat

ground truth: error:  1916296, error rate:0.479074
fake        : error:  1940613, error rate:0.485153, diff:+0.006079
original    : error:  1940082, error rate:0.485021, diff:+0.005947
------------------------------------------------ 44:25
ground truth: error:  1830173, error rate:0.457543
fake        : error:  1911310, error rate:0.477828, diff:+0.020284
original    : error:  1898349, error rate:0.474587, diff:+0.017044
------------------------------------------------ 44:26
ground truth: error:  1989569, error rate:0.497392
fake        : error:  1989776, error rate:0.497444, diff:+0.000052
original    : error:  1968189, error rate:0.492047, diff:-0.005345
------------------------------------------------ 45:26
ground truth: error:  1014363, error rate:0.253591
fake        : error:   803509, error rate:0.200877, diff:-0.052713
original    : error:        0, error rate:0.000000, diff:-0.253591
------------------------------------------------ 46:26
ground truth: error:  1763357, error rat

In [14]:
# >, !=
OUTPUT_TMP_PATH = "/Users/luminshen/Desktop/模型/new gan/T-3>9>/Ticket_OI_11_00_fake.csv"

OUTPUT_PATH = ""
ORIGINAL_PATH = "/Users/luminshen/Desktop/模型/original gan/Ticket/Ticket_OI_11_00_fake.csv"

def verify_rule(path, ratio, c1, c2, name):
    if ratio == -1:
        file = list(np.array(pd.read_csv(path, sep=',')))
    else:
        file = list(np.array(pd.read_csv(path, sep=';')))
    col_1 = c1
    col_2 = c2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] == file[j][col_1] and file[i][col_2] == file[j][col_2]:
                t -= 1
                if t < 0:
                    count += 1

    error_rate = count / len(file)**2
    if ratio == -1:
        print("ground truth: error: {:8.0f}, error rate:{:8.6f}".format(count, count / len(file)**2))
        return count / len(file)**2    
    
    else:    
        if error_rate - ratio > 0:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:+{:8.6f}".format(name, count, error_rate, error_rate - ratio))
        else:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:{:8.6f}".format(name, count, error_rate, error_rate - ratio))

        return error_rate - ratio, count



ratio = verify_rule(GROUND_TRUTH_PATH, -1, 0, 9, "")
verify_rule(OUTPUT_TMP_PATH, ratio, 0, 9, "fake")
verify_rule(ORIGINAL_PATH, ratio, 0, 9, "original")
print("------------------------------------------------")

count_fake = 0
count_original = 0
sum_fake = 0
sum_original = 0

for c1 in range(10):
    for c2 in range(10):
        if c1 == c2:
            continue
        ratio = verify_rule(GROUND_TRUTH_PATH, -1, c1, c2 , "")
        diff_fake, error_new = verify_rule(OUTPUT_TMP_PATH, ratio, c1, c2, "fake")
        diff_original, error_original = verify_rule(ORIGINAL_PATH, ratio, c1, c2, "original")
        if error_new > 0:
            sum_fake += abs(error_new)
        if error_original > 0:
            sum_original += abs(error_original)
        if abs(diff_fake) > abs(diff_original):
            count_original += 1
        else:
            count_fake += 1
        print("------------------------------------------------ {}:{}".format(count_fake, count_original))

print("fake: {}, original: {}, ratio: {}, fake_diff: {}. original_diff: {}".format(count_fake, count_original, count_fake/count_original, sum_fake, sum_original))

ground truth: error:   237200, error rate:0.059300
fake        : error:   109444, error rate:0.027361, diff:-0.031939
original    : error:   205760, error rate:0.051440, diff:-0.007860
------------------------------------------------
ground truth: error:   473286, error rate:0.118321
fake        : error:    49314, error rate:0.012328, diff:-0.105993
original    : error:    48130, error rate:0.012032, diff:-0.106289
------------------------------------------------ 1:0
ground truth: error:   473286, error rate:0.118321
fake        : error:    45352, error rate:0.011338, diff:-0.106983
original    : error:    41446, error rate:0.010361, diff:-0.107960
------------------------------------------------ 2:0
ground truth: error:   473286, error rate:0.118321
fake        : error:    45716, error rate:0.011429, diff:-0.106893
original    : error:    19578, error rate:0.004895, diff:-0.113427
------------------------------------------------ 3:0
ground truth: error:    20062, error rate:0.005015
f

original    : error:     6260, error rate:0.001565, diff:-0.000375
------------------------------------------------ 12:22
ground truth: error:    20862, error rate:0.005215
fake        : error:     2890, error rate:0.000723, diff:-0.004493
original    : error:     5462, error rate:0.001365, diff:-0.003850
------------------------------------------------ 12:23
ground truth: error:   237200, error rate:0.059300
fake        : error:   126090, error rate:0.031523, diff:-0.027777
original    : error:   186954, error rate:0.046739, diff:-0.012561
------------------------------------------------ 12:24
ground truth: error:    20062, error rate:0.005015
fake        : error:     8472, error rate:0.002118, diff:-0.002897
original    : error:    13572, error rate:0.003393, diff:-0.001622
------------------------------------------------ 12:25
ground truth: error:    20062, error rate:0.005015
fake        : error:    11896, error rate:0.002974, diff:-0.002041
original    : error:    12010, error rat

ground truth: error:     7760, error rate:0.001940
fake        : error:     2262, error rate:0.000566, diff:-0.001375
original    : error:     2592, error rate:0.000648, diff:-0.001292
------------------------------------------------ 15:54
ground truth: error:     7760, error rate:0.001940
fake        : error:     2444, error rate:0.000611, diff:-0.001329
original    : error:     3320, error rate:0.000830, diff:-0.001110
------------------------------------------------ 15:55
ground truth: error:     7760, error rate:0.001940
fake        : error:     2036, error rate:0.000509, diff:-0.001431
original    : error:     2486, error rate:0.000621, diff:-0.001319
------------------------------------------------ 15:56
ground truth: error:     7758, error rate:0.001940
fake        : error:     6142, error rate:0.001535, diff:-0.000404
original    : error:    12914, error rate:0.003229, diff:+0.001289
------------------------------------------------ 16:56
ground truth: error:    20862, error rat